![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**1- Make sure that your Python version is 3.3 or higher**

In [1]:
from platform import python_version
print (python_version())

3.9.13


**2- Install SASPy**

In [2]:
pip install saspy

Note: you may need to restart the kernel to use updated packages.


**3- Make sure that the configuration file "sascfg_personal.py" is correctly created**

In [3]:
import saspy, os
print(saspy.__file__.replace('__init__.py', 'sascfg_personal.py'))

C:\Users\CarlosD\anaconda3\lib\site-packages\saspy\sascfg_personal.py


**4- Establish Connection (Need to do this step each time you use SASPy)**

In [4]:
import saspy
sas_session = saspy.SASsession()
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 21748



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = C:\Users\CarlosD\anaconda3\lib\site-packages\saspy\sascfg_personal.py
WORK Path             = /saswork/SAS_work48B700015B41_odaws01-usw2.oda.sas.com/SAS_work4A7000015B41_odaws01-usw2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.0
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 88897


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [5]:
%%SAS sas_session

libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [6]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [7]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
640278,2640279.0,GRAHAM,PETER,0.0,90.0,44900.0,University / College,Downtown,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,20.0
568857,2568858.0,ALLRED,ALDA,1.0,41.0,69200.0,University / College,City,9.0,4.0,...,3.0,6.0,30.0,10.0,10.0,0.0,0.0,0.0,1.0,10.0


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [8]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['MinGift']].fillna(value=0)


In [9]:
data_merge.loc[data_merge["Education"] == "Elementary", "Education"] = 0

data_merge.loc[data_merge["Education"] == "High School", "Education"] = 1

data_merge.loc[data_merge["Education"] == "University / College", "Education"] = 2

dummies = pd.get_dummies(data_merge['City'], drop_first = True)
#dummies.head()

# Añadimos las variables binarias al DataFrame
data_merge = pd.concat([data_merge, dummies], axis = 1)

# Eliminamos la vairable original
data_merge = data_merge.drop(columns=['City'])


data_merge.sample(3)

,ID,LastName,FirstName,Woman,Age,Salary,Education,SeniorList,NbActivities,Referrals,...,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear,Downtown,Rural,Suburban
585405,2585406.0,PALMERI,CATHERINE,1.0,30.0,4600.0,2,9.0,1.0,0.0,...,40.0,40.0,0.0,0.0,0.0,1.0,25.0,1,0,0
518198,2518199.0,HART,HERMAN,0.0,25.0,181600.0,2,7.0,2.0,3.0,...,20.0,20.0,0.0,0.0,0.0,1.0,20.0,0,0,1
733389,2733390.0,BAGWELL,MARIA,1.0,57.0,159400.0,2,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,40.0,0,1,0


## Data Partition

In [10]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.5,random_state=5678) # you can change the percentage
train.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,SeniorList,NbActivities,Referrals,...,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear,Downtown,Rural,Suburban
318523,2318524.0,KILBURN,DON,0.0,76.0,19800.0,1,5.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,20.0,0,1,0
457094,2457095.0,FLAVIN,STEPHANIE,1.0,55.0,20700.0,0,6.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,50.0,0,1,0



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [11]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[ 53.55458359  15.73544679 123.69913525 ...  25.09809579  69.16366951
  63.00596767]


In [12]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

63.99425787440993
54380.25168908581
233.1957368587295


## Regression Tree Model（Py）

In [13]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[ 51.71708524  27.27249619 139.59760274 ...  52.95749142  46.50993087
  67.88028169]


In [14]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

63.30760878486594
54446.858003692214
233.3385051886898


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


## Scoring New Data

### Prepare data for scoring

In [15]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [16]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_contact[['MinGift']] = scoring_data_contact[['MinGift']].fillna(value=0)
scoring_data_contact[['MaxGift']] = scoring_data_contact[['MinGift']].fillna(value=0)
scoring_data_contact[['TotalGift']] = scoring_data_contact[['MinGift']].fillna(value=0)

scoring_data_contact.loc[scoring_data_contact["Education"] == "Elementary", "Education"] = 0

scoring_data_contact.loc[scoring_data_contact["Education"] == "High School", "Education"] = 1

scoring_data_contact.loc[scoring_data_contact["Education"] == "University / College", "Education"] = 2

dummies = pd.get_dummies(scoring_data_contact['City'], drop_first = True)
#dummies.head()

# Añadimos las variables binarias al DataFrame
scoring_data_contact = pd.concat([scoring_data_contact, dummies], axis = 1)

# Eliminamos la vairable original
scoring_data_contact = scoring_data_contact.drop(columns=['City'])

#scoring_data_contact.head()

X = scoring_data_contact[['Age', 'Salary','Contact','MinGift', 'AmtLastYear','GaveLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 

regr_predict_contact=regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

C:\Users\CarlosD\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,ID,AmtContact
0,2000001.0,70.433618
1,2000002.0,49.584990
2,2000003.0,57.504044
3,2000004.0,28.597231
4,2000005.0,60.276762


### Predict 'amount given' for members who were not contacted

In [17]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_nocontact[['MinGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)
scoring_data_nocontact[['MaxGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)
scoring_data_nocontact[['TotalGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)

scoring_data_nocontact.loc[scoring_data_nocontact["Education"] == "Elementary", "Education"] = 0

scoring_data_nocontact.loc[scoring_data_nocontact["Education"] == "High School", "Education"] = 1

scoring_data_nocontact.loc[scoring_data_nocontact["Education"] == "University / College", "Education"] = 2

dummies = pd.get_dummies(scoring_data_nocontact['City'], drop_first = True)
#dummies.head()

# Añadimos las variables binarias al DataFrame
scoring_data_nocontact = pd.concat([scoring_data_nocontact, dummies], axis = 1)

# Eliminamos la vairable original
scoring_data_nocontact = scoring_data_nocontact.drop(columns=['City'])

#scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age', 'Salary','Contact','MinGift','AmtLastYear', 'GaveLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 

regr_predict_nocontact=regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

C:\Users\CarlosD\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,ID,AmtNoContact
0,2000001.0,74.649167
1,2000002.0,53.800540
2,2000003.0,61.719594
3,2000004.0,32.812781
4,2000005.0,64.492312


In [18]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,70.433618,74.649167
1,2000002.0,49.584990,53.800540
2,2000003.0,57.504044,61.719594


## Exporting Results to a CSV File

In [19]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [20]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.